In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.1 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import requests
import json
from datetime import datetime
from tqdm import tqdm
import time
import nltk
import numpy as np
nltk. download('punkt')
from transformers import pipeline

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
PROJECT_PATH = "/content/drive/MyDrive/projects/interesting_visuals/company_sentiment/"

label_dict = {
    "LABEL_0": -1,
    "LABEL_1": 0,
    "LABEL_2": 1,
}

sentiment_pipeline = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment", device=0)
tokenizer_kwargs = {'max_length':512, 'truncation':True}
companies = ["apple", "google", "microsoft", "amazon", "tesla", "meta", "facebook", "alphabet", "netflix"]
days_back = 360
subreddit = "cscareerquestions"
fields = "body,author,score,created_utc,id,link_id,parent_id,permalink,subreddit"
size = 100

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

In [32]:
def get_api_text(company_name, subreddit, size, after, fields):
    url = f"https://api.pushshift.io/reddit/search/comment/?q={company_name}&subreddit={subreddit}&size={size}&after={after}d&before{after+1}d&fields={fields}"

    while True:
        r = requests.get(url)
        if "429 Too Many Requests" in r.text:
            print("Sleep...")
            time.sleep(5)
            continue
        else:
            break

    time.sleep(0.1)

    return r.text

def get_relevant_text(text, company):
    sent_list = nltk.tokenize. sent_tokenize(text) 

    relevant_sentences = []
    for sent in sent_list:
        sent_len = len(sent.split())
        if company.lower() in sent.lower() and sent_len >= 3 and sent_len < 128:
            relevant_sentences.append(sent)

    return " ".join(relevant_sentences)

def sentiment(text):
    return label_dict[sentiment_pipeline(text, **tokenizer_kwargs)[0]["label"]]



text = "I like working. Amazon is shit. But I love Google."
assert sentiment(get_relevant_text(text, "Amazon")) < 0
assert sentiment(get_relevant_text(text, "Google")) > 0

In [39]:
for company in companies:
    print(company)

    year_results = []
    for after in tqdm(range(1, days_back)):
        text = get_api_text(company, subreddit, size, after, fields)
        results = json.loads(text)
        if len(results["data"]) == 0: continue

        results_df = pd.DataFrame(results["data"])
        results_df["time"] = results_df["created_utc"].apply(lambda t: datetime.fromtimestamp(t))
        year_results.append(results_df)
        
    year_results = pd.concat(year_results).reset_index(drop=True)
    year_results = year_results[year_results["score"] > -10]
    year_results["relevant_text"] = year_results["body"].apply(lambda text: get_relevant_text(text, company))
    year_results["sentiment"] = year_results["relevant_text"].apply(lambda text: sentiment(text))

    year_results.to_json(PROJECT_PATH + f"data/{company}.json")

    display(year_results.head())

apple


  2%|▏         | 7/359 [00:03<03:21,  1.74it/s]

Sleep...


  6%|▌         | 20/359 [00:16<03:38,  1.55it/s]

Sleep...


...

Sleep...


 96%|█████████▌| 345/359 [06:47<00:08,  1.61it/s]

Sleep...


100%|██████████| 359/359 [07:03<00:00,  1.18s/it]
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


,author,body,created_utc,id,link_id,parent_id,permalink,score,subreddit,time,relevant_text,sentiment
0,wfh-without-pants,"Google, AWS, Netflix, Shopify, Brightcove. Als...",1657159336,if5kv53,t3_vszziw,t1_if4tu0x,/r/cscareerquestions/comments/vszziw/31yo_full...,1,cscareerquestions,2022-07-07 02:02:16,"Google, AWS, Netflix, Shopify, Brightcove.",1
1,WeakDescription4,"Close relative is L6 equivalent at Netflix, 10...",1657185876,if6qa4l,t3_vswzpf,t3_vswzpf,/r/cscareerquestions/comments/vswzpf/those_of_...,1,cscareerquestions,2022-07-07 09:24:36,"Close relative is L6 equivalent at Netflix, 10...",0
2,jellybeaning,my cousin got a $250k offer at Netflix right o...,1657024179,iexnuie,t3_vrmm8z,t3_vrmm8z,/r/cscareerquestions/comments/vrmm8z/is_it_gen...,1,cscareerquestions,2022-07-05 12:29:39,my cousin got a $250k offer at Netflix right o...,1
3,mhilliker,Netflix had RSUs? Since when?,1657028064,iexvb1n,t3_vrmdov,t1_iew20mz,/r/cscareerquestions/comments/vrmdov/opinion_o...,1,cscareerquestions,2022-07-05 13:34:24,Netflix had RSUs?,0
4,netstudent,Just Netflix while you were supposed to work a...,1657055805,iezrnho,t3_vs3hm1,t3_vs3hm1,/r/cscareerquestions/comments/vs3hm1/asking_ma...,1,cscareerquestions,2022-07-05 21:16:45,Just Netflix while you were supposed to work a...,0
